<a href="https://colab.research.google.com/github/ahsank/runml/blob/main/TrendTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata

In [7]:
ticker = 'SPY'
data = findata.fetch_data(ticker)
data.tail(1)

,open,high,low,close,adjclose,volume,ticker
2022-12-23,379.649994,383.059998,378.029999,382.910004,382.910004,59761100,SPY


In [22]:
tdata = data.tail(1000)
df = (tdata['adjclose']
     .rolling(window=200, min_periods=1)
     .mean()
     .to_frame('ma'))
#          .dropna()
newdata = df.join(data)
newdata.head(10) 

,ma,open,high,low,close,adjclose,volume,ticker
2019-01-08,240.227051,256.820007,257.309998,254.000000,256.769989,240.227051,102512600,SPY
2019-01-09,240.788406,257.559998,258.910004,256.190002,257.970001,241.349762,95006600,SPY
2019-01-10,241.259313,256.260010,259.160004,255.500000,258.880005,242.201126,96823900,SPY
2019-01-11,241.518154,257.679993,259.010010,257.029999,258.980011,242.294678,73858100,SPY
2019-01-14,241.377814,256.859985,258.299988,256.410004,257.399994,240.816452,70908200,SPY
2019-01-15,241.744245,257.820007,260.700012,257.809998,260.350006,243.576401,85208300,SPY
2019-01-16,242.090186,260.829987,261.970001,260.600006,260.980011,244.165833,77636700,SPY
2019-01-17,242.581196,260.010010,263.920013,259.959991,262.959991,246.018265,96118400,SPY
2019-01-18,243.326926,264.980011,266.980011,263.000000,266.459991,249.292770,127900300,SPY
2019-01-22,243.586699,264.820007,265.059998,261.059998,262.859985,245.924652,115531200,SPY


In [23]:
final_price = newdata['adjclose'].iloc[-1]
buy_profit  = lambda current, ma: final_price - current if current > ma else 0
sell_profit = lambda current, ma: current - final_price if current < ma else 0
buy_dollar = lambda current, ma: current if current > ma else 0
sell_dollar = lambda current, ma: -current if current < ma else 0

fns = {'buy_profit': buy_profit,
       'sell_profit': sell_profit,
       'buy_dollar': buy_dollar,
       'sell_dollar': sell_dollar}

for col, fn in fns.items():
  # since we don't have profit for last sequence, add 0's
  newdata[col] = list(map(fn,
                            newdata["adjclose"],
                            newdata["ma"]))
                  

newdata.tail(10)


,ma,open,high,low,close,adjclose,volume,ticker,buy_profit,sell_profit,buy_dollar,sell_dollar
2022-12-12,398.884217,394.109985,398.950012,393.410004,398.950012,397.126404,75405800,SPY,0.000000,14.216400,0.000000,-397.126404
2022-12-13,398.736582,410.220001,410.489990,399.070007,401.970001,400.132599,123782500,SPY,-17.222595,0.000000,400.132599,0.000000
2022-12-14,398.608876,401.609985,405.500000,396.309998,399.399994,397.574341,108111300,SPY,0.000000,14.664337,0.000000,-397.574341
2022-12-15,398.393624,394.299988,395.250000,387.890015,389.630005,387.848999,117705900,SPY,0.000000,4.938995,0.000000,-387.848999
2022-12-16,398.166203,385.179993,386.579987,381.040009,383.269989,383.269989,119764500,SPY,0.000000,0.359985,0.000000,-383.269989
2022-12-19,397.939949,383.470001,383.820007,378.279999,380.019989,380.019989,79878100,SPY,0.000000,-2.890015,0.000000,-380.019989
2022-12-20,397.778979,379.230011,382.230011,377.850006,380.540009,380.540009,74427200,SPY,0.000000,-2.369995,0.000000,-380.540009
2022-12-21,397.662104,383.250000,387.410004,382.690002,386.230011,386.230011,78167400,SPY,0.000000,3.320007,0.000000,-386.230011
2022-12-22,397.462771,383.049988,386.209991,374.769989,380.720001,380.720001,100120900,SPY,0.000000,-2.190002,0.000000,-380.720001
2022-12-23,397.283884,379.649994,383.059998,378.029999,382.910004,382.910004,59761100,SPY,0.000000,0.000000,0.000000,-382.910004


In [24]:
total_buy_profit = newdata["buy_profit"].sum()
total_sell_profit = newdata['sell_profit'].sum()
total_buy_dollar = newdata['buy_dollar'].sum()
total_sell_dollar = newdata['sell_dollar'].sum()
total_return = (total_buy_profit + total_sell_profit)/(total_buy_dollar + total_sell_dollar)
total_return


0.13477183089695693